In [2]:
import os

from tqdm.notebook import tqdm, trange
# from tqdm import tqdm_notebook as tqdm
import time    # to be used in loop iterations
import sqlite3
import pandas as pd

In [3]:
project = "D:/Routelines/"
cif_path = project + 'data/'
db_path = project + 'db/'

cif ='Bus_1.cif'
cif_file = cif_path + cif

print(cif_path)
print(cif_file)
print(db_path)

D:/Routelines/data/
D:/Routelines/data/Bus_1.cif
D:/Routelines/db/


In [ ]:
# Simple loop
for i in range(100):
    pass

# Loop with a progress bar
for i in trange(100):
    time.sleep(0.01)

In [ ]:
# Loop with a progres bar
for i in tqdm(range(100)):
    time.sleep(0.01)

In [4]:
outer_level = list(range(2))
inner_level = list(range(100))
for _ in tqdm(outer_level, desc='Outer Level'):
    for number in tqdm(inner_level, desc='Inner Level'):
        time.sleep(0.01)

Outer Level:   0%|          | 0/2 [00:00<?, ?it/s]

Inner Level:   0%|          | 0/100 [00:00<?, ?it/s]

Inner Level:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
p_bar = tqdm(range(10))
for number in p_bar:
    time.sleep(0.5)
    p_bar.set_description(f'Working on "{number}"')

#### pandas dataframe

In [ ]:
# pd.read_csv(cif_file, names=['CODE'], header=None, sep='!')

In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

project = "D:/Routelines/"
cif_path = project + 'data/'
db_path = project + 'db/'
DB_FILENAME = db_path + 'progresbar_test.db'

def chunker(seq, size):
    # from http://stackoverflow.com/a/434328
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def insert_with_progress(df, dbfile):
    con = sqlite3.connect(dbfile)
    chunksize = int(len(df) / 10) # 10%
    print("len(df) = ", len(df))
    print("chunksize =", chunksize)
    with tqdm(total=len(df)) as pbar:
        for i, cdf in enumerate(chunker(df, chunksize)):
            print("i=", i)
            print("cdf=", cdf)
            replace = "replace" if i == 0 else "append"
            cdf.to_sql(con=con, name="MLS", if_exists='append', index=False)
            pbar.update(chunksize)
            
df = pd.DataFrame({'a': range(0,390975)})
insert_with_progress(df, DB_FILENAME)

len(df) =  390975
chunksize = 39097


  0%|          | 0/390975 [00:00<?, ?it/s]

i= 0
cdf=            a
0          0
1          1
2          2
3          3
4          4
...      ...
39092  39092
39093  39093
39094  39094
39095  39095
39096  39096

[39097 rows x 1 columns]
i= 1
cdf=            a
39097  39097
39098  39098
39099  39099
39100  39100
39101  39101
...      ...
78189  78189
78190  78190
78191  78191
78192  78192
78193  78193

[39097 rows x 1 columns]
i= 2
cdf=              a
78194    78194
78195    78195
78196    78196
78197    78197
78198    78198
...        ...
117286  117286
117287  117287
117288  117288
117289  117289
117290  117290

[39097 rows x 1 columns]
i= 3
cdf=              a
117291  117291
117292  117292
117293  117293
117294  117294
117295  117295
...        ...
156383  156383
156384  156384
156385  156385
156386  156386
156387  156387

[39097 rows x 1 columns]
i= 4
cdf=              a
156388  156388
156389  156389
156390  156390
156391  156391
156392  156392
...        ...
195480  195480
195481  195481
195482  195482
195483  195483
195484  1

In [ ]:
chunk_size=1000000
chunks = []
loop = True

reader = pd.read_csv(cif_file, names=['CODE'], header=None, sep='!', iterator=True)
while loop:
    try:
        chunk = reader.get_chunk(chunk_size)
        chunks.append(chunk)
    except StopIteration:
        loop = False
        print("Iteration is stopped")

pbar.close()
df_ac = pd.concat(chunks, ignore_index=True)

df_ac

In [ ]:
import os
import mmap
from tqdm.notebook import tqdm, trange
import pandas as pd

def mapcount(filename):
    f = open(filename, "r+")
    buf = mmap.mmap(f.fileno(), 0)
    lines = 0
    readline = buf.readline
    while readline():
        lines += 1
    return lines

print(f'CIF file size is {round(os.stat(cif_file).st_size / (1024 * 1024), 2)} MB')   
print("Counting number of lines...")
count_lines = mapcount(cif_file)
print(f'{count_lines} lines in the file!')

chunksize = int(count_lines / 46)
print(chunksize)

chunks = []

with pd.read_csv(cif_file, names=['CODE'], header=None, sep='!', iterator=True) as reader:
    for i in tqdm(range(count_lines)):
        chunk = reader.get_chunk(chunksize)
        chunks.append(chunk)

df_ac = pd.concat(chunks, ignore_index=True)

In [ ]:
import os
import mmap
from tqdm.notebook import tqdm, trange
import pandas as pd

def mapcount(filename):
    f = open(filename, "r+")
    buf = mmap.mmap(f.fileno(), 0)
    lines = 0
    readline = buf.readline
    while readline():
        lines += 1
    return lines

print(f'CIF file size is {round(os.stat(cif_file).st_size / (1024 * 1024), 2)} MB')   
print("Counting number of lines...")
count_lines = mapcount(cif_file)
print(f'{count_lines} lines in the file!')

chunksize = int(count_lines / 46)
print(chunksize)

chunks = []

chunks = pd.read_csv(cif_file, names=['CODE'], header=None, sep='!', iterator=True, chunksize=chunksize)

df_ac = pd.concat(chunks, ignore_index=True)

In [ ]:
df_ac

In [ ]:
import os
import sys
import pandas as pd
from tqdm import tqdm

fn = cif_file

temp = pd.read_csv(fn, nrows=20)
N = len(temp.to_csv(index=False))
t = int(os.path.getsize(fn)/N*20/10**5) + 1

df = temp[:0]

with tqdm(total = t, file = sys.stdout) as pbar:
    for i,chunk in enumerate(pd.read_csv(fn, chunksize=10**5, low_memory=False, sep='?')):
        df = df.append(chunk)
        pbar.set_description('Importing: %d' % (1 + i))
        pbar.update(1)

In [ ]:
import os
import sys
import pandas as pd
from tqdm.notebook import tqdm
import mmap
import math

project = "D:/Routelines/"
# project = "C:/temp/"
cif_path = project + 'data/'
db_path = project + 'db/'

# cif ='ATCO_Bus.cif'
cif ='Bus_1.cif'
cif_file = cif_path + cif

# print(cif_path)
print(cif_file)
# print(db_path)

def mapcount(filename):
    f = open(filename, "r+")
    buf = mmap.mmap(f.fileno(), 0)
    lines = 0
    readline = buf.readline
    while readline():
        lines += 1
    return lines

print(f'CIF file size is {round(os.stat(cif_file).st_size / (1024 * 1024), 2)} MB')   
print("Counting number of lines...")
count_lines = mapcount(cif_file)
print(f'{count_lines} lines in the file!')

chunk_size=1000000
chunks = []

loops = math.ceil(count_lines/chunk_size)
# print(loops)

i=0
with tqdm(total = loops, file = sys.stdout) as pbar:
    reader = pd.read_csv(cif_file, names=['CODE'], header=None, sep='!', iterator=True)
    while i <= loops:
        try:
            i+=1
            chunk = reader.get_chunk(chunk_size)
            chunks.append(chunk)
#             pbar.set_description('Lines imported: %d' % (1*chunk_size + i*chunk_size))
            pbar.set_description('Importing CIF data to Pandas dataframe')
            pbar.update(1)
        except StopIteration:
            loop = False
            df_ac = pd.concat(chunks, ignore_index=True)
            print("CIF data imported!")

pbar.close()

df_ac